In [26]:
import sys, os, cv2, torch
sys.path.append(f"{os.getcwd()}")
from PIL import Image
from torchvision.transforms import transforms
from yacs.config import CfgNode as CN
from style_transfer.networks import AdaIN, AdaINConfig
from torchvision.utils import save_image

In [27]:
def transform(contentPath, stylePath):
    config = AdaINConfig.create("style_transfer/config/adain.yaml")
    config.defrost()
    config.preserve_color = False
    config.alpha = 1.0
    config.save_dir = "../.output/results"
    config.freeze()
    network = AdaIN(config)
    network.loadModel({
        "vgg": "../../../Models/AdaIN/vgg_normalised.pth",
        "decoder": "../../../Models/AdaIN/decoder.pth"
    })
    with open(contentPath, 'rb') as file:
        content = Image.open(file)
        content.load()
        
    with open(stylePath, 'rb') as file:
        style = Image.open(file)
        style.load()
        
    composite = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(config.content_size),
    ])
    
    content = composite(content)
    style = composite(style)
    
    styledImage = network.transformTo(content, style)

    styledImage = styledImage[0].detach().cpu().numpy()
    styledImage = styledImage.transpose(1,2,0)
    styledImage = cv2.cvtColor(styledImage, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(config.save_dir, "test_adain.png"), styledImage * 255)

In [28]:
# train()
transform("../../../Datasets/custom/photo.png", "../../../Datasets/custom/style.png")

1.0


d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
